# Recurrent Neural Networks

* [Ultimate Guide to Recurrent Neural Networks](https://www.superdatascience.com/blogs/the-ultimate-guide-to-recurrent-neural-networks-rnn)
* Used for `Time Series` data analysis
* Weights like in ANNs represent long-term memory
  * Similar to the temporal lobe of the brain
* RNNs are like short-term memory
  * Similar to the frontal lobe of the brain
* Can be thought of as a ANN squashed together (or looking from another dimension) and inverted with a temporal loop with in the hidden layer
  * Temporal loop meaning it feeds data back into itself in addition to the output
  * Each blue circle representing a whole layer of neurons <img src="img/41_blog_image_10.png">
* Types of RNNs
  * 1:Many - One input and many outputs
    * e.g. An image input with a descriptive output of the image. "black and white dog jumps over bar"
  * Many:1 - Many inputs and one output
    * e.g. Sentiment analysis. "Thanks for a great party!" - positive score: 86%
  * Many:Many - Many inputs to many outputs
    * e.g. Google translator to maintain gender specific information
    * e.g. Movie subtitles - need context to understand plot
* `Vanishing gradient problem`
  * As weights are updated backward through the network if the value (error) being applied to the weights (Wrec) is too small then the weights get too small to be relevant (vanishign) or if too big then the weights get too large (exploding) and invalidates the entire network
* `Long short-term memory` (LSTM)
  *  Solution to vanishing gradient problem <img src="img/41_blog_image_22.png" >
  *  pointwise operations are like valves (X) controlled by neural network layers
     *  forget valve
     *  memory valve
     *  output valve
  *  neural network layer
     *  sigmoid
     *  tanh
  *  Variations
     *  Peep-hole - adding additional information to neural network layers (sigmoid)
     *  Combine decisioning between valves
     *  Gated recurring units (GRUs)
  *  Reading
     *  [LSTM: A Search Space Odyssey](https://arxiv.org/pdf/1503.04069.pdf) by Klaus Greff et al. (2015) 

## A stacked LSTM model with Dropout Regularization

In [1]:
from os.path import dirname, abspath, join, curdir

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Data preprocessing

In [2]:
# Import the training set
datapath = join(dirname(dirname(abspath(curdir))), "data", "raw")

dataset_train = pd.read_csv(join(datapath, "googl_stock_prices_2012-2016.csv"))
train_set = dataset_train.iloc[:, 1:2].values
train_set, train_set.shape

(array([[325.25],
        [331.27],
        [329.83],
        ...,
        [793.7 ],
        [783.33],
        [782.75]]),
 (1258, 1))

In [3]:
# Apply recommended normalization scaling
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range=(0, 1), copy=True)
train_set_scaled = sc.fit_transform(train_set) # Fit with data min/max then transforms
train_set_scaled, train_set_scaled.size

(array([[0.08581368],
        [0.09701243],
        [0.09433366],
        ...,
        [0.95725128],
        [0.93796041],
        [0.93688146]]),
 1258)

In [4]:
# Create data structure with 60 timesteps (past info) and 1 output
X_train = []
y_train = []

for i in range(60, train_set_scaled.size):
    X_train.append(train_set_scaled[i-60:i, 0])
    y_train.append(train_set_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

In [5]:
X_train

array([[0.08581368, 0.09701243, 0.09433366, ..., 0.07846566, 0.08034452,
        0.08497656],
       [0.09701243, 0.09433366, 0.09156187, ..., 0.08034452, 0.08497656,
        0.08627874],
       [0.09433366, 0.09156187, 0.07984225, ..., 0.08497656, 0.08627874,
        0.08471612],
       ...,
       [0.92106928, 0.92438053, 0.93048218, ..., 0.95475854, 0.95204256,
        0.95163331],
       [0.92438053, 0.93048218, 0.9299055 , ..., 0.95204256, 0.95163331,
        0.95725128],
       [0.93048218, 0.9299055 , 0.93113327, ..., 0.95163331, 0.95725128,
        0.93796041]])

In [6]:
y_train

array([0.08627874, 0.08471612, 0.07454052, ..., 0.95725128, 0.93796041,
       0.93688146])

In [7]:
# Reshaping to add new dimension to train data, the new dimension can be any 
# number of additional indicators but for now all we're using is the open price
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train, X_train.shape

(array([[[0.08581368],
         [0.09701243],
         [0.09433366],
         ...,
         [0.07846566],
         [0.08034452],
         [0.08497656]],
 
        [[0.09701243],
         [0.09433366],
         [0.09156187],
         ...,
         [0.08034452],
         [0.08497656],
         [0.08627874]],
 
        [[0.09433366],
         [0.09156187],
         [0.07984225],
         ...,
         [0.08497656],
         [0.08627874],
         [0.08471612]],
 
        ...,
 
        [[0.92106928],
         [0.92438053],
         [0.93048218],
         ...,
         [0.95475854],
         [0.95204256],
         [0.95163331]],
 
        [[0.92438053],
         [0.93048218],
         [0.9299055 ],
         ...,
         [0.95204256],
         [0.95163331],
         [0.95725128]],
 
        [[0.93048218],
         [0.9299055 ],
         [0.93113327],
         ...,
         [0.95163331],
         [0.95725128],
         [0.93796041]]]),
 (1198, 60, 1))

### Building the RNN

In [8]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

2023-08-25 22:35:57.664897: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# Initialize the RNN
rnn = Sequential()

In [10]:
# Add the first LSTM layer with dropout regularization to prevent overfitting

# units = # of neurons, need high dimensionality to capture upward/downward trends
# return_sequences = True when there will be multiple LSTM layers
rnn.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
rnn.add(Dropout(0.2)) # ignore 20% of neurons during each training iteration

In [11]:
# Add second LSTM/Dropout layers
rnn.add(LSTM(units=50, return_sequences=True))
rnn.add(Dropout(0.2))

# Add third LSTM/Dropout layers
rnn.add(LSTM(units=50, return_sequences=True))
rnn.add(Dropout(0.2))

# Add fourth LSTM/Dropout layers
rnn.add(LSTM(units=50))
rnn.add(Dropout(0.2))

In [12]:
# Add the output layer
rnn.add(Dense(units=1))

In [13]:
# Compile the RNN
rnn.compile(optimizer="adam", loss="mean_squared_error")

In [14]:
# Fitting the RNN to the training set
rnn.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
38/38 [==============================] - 8s 52ms/step - loss: 0.0412
Epoch 2/100
38/38 [==============================] - 2s 53ms/step - loss: 0.0073
Epoch 3/100
38/38 [==============================] - 2s 52ms/step - loss: 0.0051
Epoch 4/100
38/38 [==============================] - 2s 52ms/step - loss: 0.0051
Epoch 5/100
38/38 [==============================] - 2s 52ms/step - loss: 0.0048
Epoch 6/100
38/38 [==============================] - 2s 53ms/step - loss: 0.0049
Epoch 7/100
38/38 [==============================] - 2s 52ms/step - loss: 0.0047
Epoch 8/100
38/38 [==============================] - 2s 52ms/step - loss: 0.0048
Epoch 9/100
38/38 [==============================] - 2s 52ms/step - loss: 0.0044
Epoch 10/100
38/38 [==============================] - 2s 53ms/step - loss: 0.0049
Epoch 11/100
38/38 [==============================] - 2s 52ms/step - loss: 0.0041
Epoch 12/100
38/38 [==============================] - 2s 52ms/step - loss: 0.0041
Epoch 13/100
38/38 [=====

### Prediction and visualization

In [ ]:
# Load test data
